Code to get the images from the .tif file starting from the geoJSON

In [1]:
from PIL import Image
import json
import numpy as np
import os

In [ ]:
specific_date = '20240718'
tif_dir = f'./data/campaigns/{specific_date}/image_stacks'
geojson_dir = './geojson_file'
output_dir = 'extracted_shapes'
os.makedirs(output_dir, exist_ok=True)

# Image dimensions for YOLO normalization
IMAGE_WIDTH = 1616
IMAGE_HEIGHT = 1240

In [ ]:
def get_geojson_files(geojson_dir):
    """
    Get set of geoJSON file names without extensions
    """
    return {os.path.splitext(f)[0] for f in os.listdir(geojson_dir) if f.endswith('.geojson')}


def create_yolo_annotation(class_id, x_center, y_center, width, height, save_path):
    """
    Save YOLO format annotation to a text file
    """
    
    with open(save_path, 'w') as file:
        file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


def process_geojson_file(file_name):
    """
    Process a single geoJSON file and its corresponding .tif file
    """
    
    tif_path = os.path.join(tif_dir, file_name + '.tif')
    geojson_path = os.path.join(geojson_dir, file_name + '.geojson')
    
    # Check if the corresponding `.tif` file exists
    if not os.path.exists(tif_path):
        print(f"Warning: `{tif_path}` not found. Skipping this file.")
        return

    # load geoJSON
    with open(geojson_path, 'r') as file:
        geojson_data = json.load(file)

    # Open tif file and process each shape in geoJSON
    with Image.open(tif_path) as tif_img:
        for feature in geojson_data['features']:
            process_feature(feature, tif_img, file_name)

def process_feature(feature, tif_img, file_name):
    """
    Process an individual feature within a geoJSON file
    """
    
    # our classifications
    class_mapping = {"Living": 0, "Non-Living": 1, "Bubble": 2}
    
    # classification
    classification = feature['properties']['classification']['name']
    
    # Assign 'Non-Living' if classification is not found in class_mapping
    class_id = class_mapping.get(classification, class_mapping["Non-Living"])
    
    # get coordinates and frame index
    coordinates = feature['geometry']['coordinates'][0]
    frame_idx = feature['geometry']['plane']['t']

    # Access specified frame in the .tif stack
    tif_img.seek(frame_idx)
    
    # Calculate bounding box from coordinates
    x_coords = [point[0] for point in coordinates]
    y_coords = [point[1] for point in coordinates]
    min_x, max_x = int(min(x_coords)), int(max(x_coords))
    min_y, max_y = int(min(y_coords)), int(max(y_coords))
    
    # Crop and save the shape image (optional)
    frame = np.array(tif_img)
    image_filename = save_shape_image(frame, min_x, max_x, min_y, max_y, classification, file_name, frame_idx)
    
    # YOLO coordinates --> divide to get the normalization becasue of YOLO format
    x_center = (min_x + max_x) / 2 / IMAGE_WIDTH
    y_center = (min_y + max_y) / 2 / IMAGE_HEIGHT
    width = (max_x - min_x) / IMAGE_WIDTH
    height = (max_y - min_y) / IMAGE_HEIGHT
    annotation_filename = os.path.splitext(image_filename)[0] + ".txt"
    annotation_path = os.path.join(output_dir, annotation_filename)
    
    # save annotation
    create_yolo_annotation(class_id, x_center, y_center, width, height, annotation_path)


def save_shape_image(frame, min_x, max_x, min_y, max_y, classification, file_name, frame_idx):
    """
    Save cropped shape image and return its filename
    """
    
    shape_crop = frame[min_y:max_y, min_x:max_x]
    image_filename = f"{file_name}_{classification}_shape_{frame_idx}.png"
    shape_image = Image.fromarray(shape_crop)
    shape_image.save(os.path.join(output_dir, image_filename))
    return image_filename


def main():
    geojson_files = get_geojson_files(geojson_dir)
    for file_name in geojson_files:
        process_geojson_file(file_name)



In [13]:
if __name__ == "__main__":
    main()